In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
import sys
import os
import time
import pickle
import random

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from torch.optim import Adam
from sklearn import metrics
from collections import OrderedDict 

sys.path.append(os.path.abspath('..'))

import datasets
import utils.more_torch_functions as mtf
from utils.modules import Parallel, MaxLayer
from utils.custom_activations import StepActivation
from utils.custom_loss import AsymBCELoss
from utils.misc import train_model, Figures
from compiling_nn.build_odd import compile_nn
from compiling_nn.utils_odd import pickle_bdd, unpickle_bdd

seed = 2872
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

SAVE_PATH = os.path.join(os.path.abspath('..'), "backup")
PKL_PATH = os.path.join(SAVE_PATH, "bdd")
PTH_PATH = os.path.join(SAVE_PATH, "nn")
METRIC_PATH = os.path.join(SAVE_PATH, "metrics")

## Load data

In [3]:
dataset = datasets.DiabetesDataset

np_x, np_y = dataset.get_dataset(balancing=True, discretizing=True, hot_encoding=True)
x_data, y_data = torch.Tensor(np_x), torch.Tensor(np_y)
input_size = x_data.size(1)
print(x_data.size())

torch.Size([536, 29])


In [4]:
def train_eval_model(x_train, y_train, x_valid, y_valid, model, criterion, optimizer, epochs=50, metrics_average="binary"):
    train_model(x_train, y_train, model, criterion, optimizer, epochs)
    model.eval()
    pred_train = model(x_train).detach()
    pred_valid = model(x_valid).detach()

    p_train, r_train, f_train, _ = metrics.precision_recall_fscore_support(y_train, pred_train, beta=1, average=metrics_average, labels=[0,1])
    p_valid, r_valid, f_valid, _ = metrics.precision_recall_fscore_support(y_valid, pred_valid, beta=1, average=metrics_average, labels=[0,1])

    return f_train, p_train, r_train, f_valid, p_valid, r_valid

def print_eval(x_train, y_train, x_valid, y_valid, model, criterion, optimizer):
    f_train, p_train, r_train, f_valid, p_valid, r_valid = train_eval_model(x_train, y_train, x_valid, y_valid, model, criterion, optimizer)
    print(
        f"{'':<15}{'Train':^15}{'Valid':^15}",
        f"{'F1 score':<15}{f_train:^15.3f}{f_valid:^15.3f}",
        f"{'Precision':<15}{p_train:^15.3f}{p_valid:^15.3f}",
        f"{'Rappel':<15}{r_train:^15.3f}{r_valid:^15.3f}",
        sep="\n",
    )

## Network

In [5]:
class nApxNet(nn.Module):
    def __init__(self, n, hl=3) -> None:
        super().__init__()

        self.n_apx = n
        self.net = nn.Sequential(OrderedDict([
            ('nets', Parallel(OrderedDict([
                (f'apx{i}', ApproxNet(hl)) for i in range(1, self.n_apx+1)
            ]))),
            ('or_', MaxLayer()),
        ]))


    def forward(self, input):
        return self.net(input)
    
    def add_apx(self, module):
        self.n_apx += 1
        self.net.nets.add_module(f'apx{self.n_apx}', module)
    
class ApproxNet(nn.Module):
    def __init__(self, hl1):
        super().__init__()
        self.nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', StepActivation()),
            ('l2', nn.Linear(hl1,5)),
            ('a2', StepActivation()),
            ('l3', nn.Linear(5,1)),
            ('a3', StepActivation()),
        ]))        

    def forward(self, x):
        x = self.nn(x)

        return x
    
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        hl1 = 50
        hl2 = 25

        self. nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', nn.ReLU()),
            ('l2', nn.Linear(hl1,hl2)),
            ('a2', nn.ReLU()),
            ('l3', nn.Linear(hl2,1)),
            ('a3', StepActivation()),
        ]))
    
    def forward(self, x):
        x = self.nn(x)

        return x

## Exp

In [6]:
def nn_approx(x_train, y_train, x_valid, y_valid):
    exp_folder = "nn_approx"
    os.makedirs(os.path.join(PTH_PATH, exp_folder), exist_ok=True)
    d_metrics = dict()
    model = nApxNet(1, hl=5)
    criterion = AsymBCELoss() # HERE
    module = model.net.nets.apx1
    optimizer = Adam(module.parameters(), lr=1e-2, weight_decay=1e-6)

    f1t, pt, rt, f1v , pv, rv = train_eval_model(x_train, y_train, x_valid, y_valid, model, criterion, optimizer, epochs=500, metrics_average=None)

    mtf.freeze_model(model)

    module = Net()
    model.add_apx(module)
    criterion = nn.BCELoss()
    optimizer = Adam(module.parameters(), lr=1e-2, weight_decay=1e-6)

    f1t, pt, rt, f1v , pv, rv = train_eval_model(x_train, y_train, x_valid, y_valid, model, criterion, optimizer, epochs=500, metrics_average=None)

    d_metrics[('train', 'f1')] = f1t.mean()
    d_metrics[('valid', 'f1')] = f1v.mean()
    d_metrics[('train', 'precision0')] = pt[0]
    d_metrics[('valid', 'precision0')] = pv[0]
    d_metrics[('train', 'recall0')] = rt[0]
    d_metrics[('valid', 'recall0')] = rv[0]
    d_metrics[('train', 'precision1')] = pt[1]
    d_metrics[('valid', 'precision1')] = pv[1]
    d_metrics[('train', 'recall1')] = rt[1]
    d_metrics[('valid', 'recall1')] = rv[1]

    with open(os.path.join(METRIC_PATH, f"{exp_folder}.pkl"), 'wb') as f:
        pickle.dump(d_metrics, f)
    return d_metrics

In [7]:
train_index, valid_index = torch.utils.data.random_split(range(x_data.size(0)), [0.9, 0.1])

x_train, y_train = x_data[train_index], y_data[train_index]
x_valid, y_valid = x_data[valid_index], y_data[valid_index]

d_metrics = nn_approx(x_train, y_train, x_valid, y_valid)

In [8]:
print(d_metrics, sep='\n')
# train / valid
# f1 p0 r0 p1 r1

{('train', 'f1'): 0.8859327834290156, ('valid', 'f1'): 0.7239583333333333, ('train', 'precision0'): 0.9400921658986175, ('valid', 'precision0'): 0.7, ('train', 'recall0'): 0.8292682926829268, ('valid', 'recall0'): 0.6363636363636364, ('train', 'precision1'): 0.8421052631578947, ('valid', 'precision1'): 0.7575757575757576, ('train', 'recall1'): 0.9451476793248945, ('valid', 'recall1'): 0.8064516129032258}


In [16]:
def compute_metrics(pred_train, y_train, pred_valid, y_valid):
    p_train, r_train, f_train, _ = metrics.precision_recall_fscore_support(y_train, pred_train, beta=1, average=None, labels=[0,1])
    p_valid, r_valid, f_valid, _ = metrics.precision_recall_fscore_support(y_valid, pred_valid, beta=1, average=None, labels=[0,1])

    metrics_list = [
        f_train.mean(), f_valid.mean(),
        p_train[0], p_valid[0],
        r_train[0], r_valid[0],
        p_train[1], p_valid[1],
        r_train[1], r_valid[1],
    ]

    return metrics_list

net = Net()
crit = nn.BCELoss()
opt = Adam(net.parameters(), lr=1e-2, weight_decay=1e-6)

train_model(x_train, y_train, net, crit, opt, max_epoch=500)

net_pred_train = net(x_train).detach().round()
net_pred_valid = net(x_valid).detach().round()

net_metrics = compute_metrics(net_pred_train, y_train, net_pred_valid, y_valid)

print(net_metrics)

[0.9668622002675723, 0.7668621700879765, 0.967479674796748, 0.7272727272727273, 0.967479674796748, 0.7272727272727273, 0.9662447257383966, 0.8064516129032258, 0.9662447257383966, 0.8064516129032258]
